## yolov5 시작

210414

YOLOv5 : https://github.com/ultralytics/yolov5
        
- 설치(torch는 1.8 version)
> 1. git clone https://github.com/ultralytics/yolov5.git
> 2. cd yolov5
> 3. pip install -r requirements.txt

- YOLOv5x 모델 선정
- 입력이미지 640으로 작고, 그 중에 가장 성능이 좋으면서 추론시간 6ms 가능, 88M개의 파라미터 사용

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
# %pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.1+cpu (CPU)


# 1. Inference

`detect.py` runs YOLOv5 inference on a variety of sources, downloading models automatically from the [latest YOLOv5 release](https://github.com/ultralytics/yolov5/releases), and saving results to `runs/detect`. Example inference sources are:

<img src="https://user-images.githubusercontent.com/26833433/114307955-5c7e4e80-9ae2-11eb-9f50-a90e39bee53f.png" width="900"> 

In [3]:
%%time
!python detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source data/images/ --device 3
# Image(filename='runs/detect/exp/zidane.jpg', width=600)

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='3', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='data/images/', update=False, view_img=False, weights=['yolov5s.pt'])
requirements: tensorboard>=2.4.1 not found and is required by YOLOv5, attempting auto-update...
Wall time: 3.61 s


ERROR: Invalid requirement: "'tensorboard"
Traceback (most recent call last):
  File "C:\Users\User\Desktop\22_모부\yolov5_colab\utils\general.py", line 110, in check_requirements
    pkg.require(r)
  File "C:\Users\User\anaconda3\envs\venv\lib\site-packages\pkg_resources\__init__.py", line 886, in require
    needed = self.resolve(parse_requirements(requirements))
  File "C:\Users\User\anaconda3\envs\venv\lib\site-packages\pkg_resources\__init__.py", line 777, in resolve
    raise VersionConflict(dist, req).with_context(dependent_req)
pkg_resources.VersionConflict: (tensorboard 2.1.1 (c:\users\user\anaconda3\envs\venv\lib\site-packages), Requirement.parse('tensorboard>=2.4.1'))

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "detect.py", line 171, in <module>
    check_requirements(exclude=('pycocotools', 'thop'))
  File "C:\Users\User\Desktop\22_모부\yolov5_colab\utils\general.py", line 114, in check_requirements
    print(s

In [4]:
!pip install tensorboard>=2.4.1 

# 3. OD Train!

210813

In [ ]:
# Weights & Biases (optional)
%pip install -q wandb
# !wandb login 4c50afba3772e5d29da420e32e2068c28fe33a4c  # use 'wandb disabled' or 'wandb enabled' to disable or enable

In [ ]:
## 210812:18:21~(14시간), 모부 OD 데이터 1800개로 학습, 200개로 TEST 시작
# 87.7M params, 175MB weight, 640x640
# https://wandb.ai/keepsteady/YOLOv5/runs/1cturje7
!python train.py --img 640 --batch 64 \
                --epochs 1000 \
                --data mydata.yaml \
                --weights yolov5x.pt \
                --cache  # --nosave 

# 4. Test

In [ ]:
%%time
!python detect.py --weights runs/train/210813_1000/weights/best.pt \
                  --source data_mobu_210812/images/test/ \
                  --img 640 \
                  --conf 0.25 \
                  --device 3

In [ ]:
Image(filename='runs/detect/exp/c_영어_2019학년도_3월_23번.jpg', width=600)

## Local Logging

All results are logged by default to `runs/train`, with a new experiment directory created for each new training as `runs/train/exp2`, `runs/train/exp3`, etc. View train and test jpgs to see mosaics, labels, predictions and augmentation effects. Note a **Mosaic Dataloader** is used for training (shown below), a new concept developed by Ultralytics and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
Image(filename='runs/train/210813_1000/train_batch0.jpg', width=800)  # train batch 0 mosaics and labels

In [ ]:
Image(filename='runs/train/210813_1000/test_batch0_labels.jpg', width=800)  # test batch 0 labels

In [ ]:
Image(filename='runs/train/210813_1000/test_batch0_pred.jpg', width=800)  # test batch 0 predictions

Training losses and performance metrics are also logged to [Tensorboard](https://www.tensorflow.org/tensorboard) and a custom `results.txt` logfile which is plotted as `results.png` (below) after training completes. Here we show YOLOv5s trained on COCO128 to 300 epochs, starting from scratch (blue), and from pretrained `--weights yolov5s.pt` (orange).

In [ ]:
from utils.plots import plot_results 
plot_results(save_dir='runs/train/210813_1000')  # plot all results*.txt as results.png
Image(filename='runs/train/210813_1000/results.png', width=800)

In [ ]:
# PyTorch Hub
import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Images
dir = 'https://github.com/ultralytics/yolov5/raw/master/data/images/'
imgs = [dir + f for f in ('zidane.jpg', 'bus.jpg')]  # batch of images

# Inference
results = model(imgs)
results.print()  # or .show(), .save()

results.xyxy[0]  # img1 predictions (tensor)
results.pandas().xyxy[0]  # img1 predictions (pandas)